In [1]:
# adding source code parent directory
import sys
sys.path.append('..\\Src')

In [18]:
import pandas as pd
import numpy as np

In [166]:
from preprocessing.data_prep import read_data
from preprocessing.data_clean import clean_data2, clean_text2

In [3]:
data_path = "../Data/husna.json"
cols = ['text_clean', 'summary_clean', 'title_clean', 'tags']
cols_weights = {'text_clean': 0.5, 'summary_clean': 0.15, 'title_clean': 0.2, 'tags': 0.1}

In [6]:
docs_df = read_data(data_path)
docs_df_cleaned = clean_data2(docs_df)
docs_df_cleaned['content_clean'] = docs_df_cleaned['title_clean'] + " " + docs_df_cleaned['text_clean']

C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\haystack-venv\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\modaj\OneDrive\Documents\Personal\Jobs\SHAI\intern - task 3\haystack-venv\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
docs_df_cleaned['content_tokenized'] = docs_df_cleaned['text_clean'].apply(lambda x: x.split())

In [12]:
content_data = docs_df_cleaned['content_tokenized']

### Building Word Dictionary

In the next step we will build the vocabulary of the corpus in which all the unique words are given IDs and their frequency counds are also stored. You may note that we are using gensim library for building the dictionary.   In gensim, the words are referred as "tokens" adn the index of each word in the dictionary is called ID

In [85]:
from gensim import corpora, models

#creating term dictionary
%time dictionary = corpora.Dictionary(content_data)

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
#NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=4, no_above=0.2)

#list of few which which can be further removed
# stoplist = set('hello and if this can would should could tell ask stop come go')
# stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
# dictionary.filter_tokens(stop_ids)


CPU times: total: 859 ms
Wall time: 912 ms


You can see that there are 2 additional steps performed after creating the dictionary.
1. All the tokens in the dictionary which either have occurrred in less than 4 articles or have occurred in more than 20% of the articles are removed from the dictionary, as these words will not be contributing to the various themes or topics. 
2. Removing content neutral words from the dictionary and additional stopwords.

In [86]:
print(list(dictionary.items())[0])
print(dictionary[0], dictionary.token2id[dictionary[0]])

(0, '10')
10 0


In [87]:
#print top 50 items from the dictionary with their unique token-id
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 50]]
print (dict_tokens)

[[['10', 0], ['15', 1], ['42', 2], ['600', 3], ['820', 4], ['أحتاج', 5], ['أحد', 6], ['أرض', 7], ['أشار', 8], ['أكد', 9], ['أمر', 10], ['أمين', 11], ['أن', 12], ['أنتظر', 13], ['أنتقل', 14], ['أنجز', 15], ['أو', 16], ['إجراء', 17], ['إداري', 18], ['إستراتيجي', 19], ['إعداد', 20], ['إلي', 21], ['ابن', 22], ['ارحاب', 23], ['استئجار', 24], ['اكتظاظ', 25], ['ال', 26], ['الذي', 27], ['القبيلات', 28], ['بناء', 29], ['بين', 30], ['تأكد', 31], ['تبرع', 32], ['تحديد', 33], ['تحويل', 34], ['تربيه', 35], ['تسلم', 36], ['تسليم', 37], ['تعلق', 38], ['تعليم', 39], ['تنسيق', 40], ['توزيع', 41], ['توسع', 42], ['جاء', 43], ['جديد', 44], ['جمعيه', 45], ['حاجه', 46], ['حسني', 47], ['حكومي', 48], ['خلال', 49], ['دراسه', 50]]]


### Feature Extraction (Bag of Words)

A bag of words model, or BoW for short is a way of extracting features from text for use in modelling, such as with machine learning algorithms. It is a representation of tet that describes teh occurence of words within a document. It involves two things

1. A vocabulary of known words
2. A measure of the presence of known words

The doc2bow method of dictionary, iterates through all the words in the text, if the word already exists in the corpus, it increments the frequency count, other wise it inserts the word into the corpus and sets it freqeuncy count to 1

In [88]:
# dictionary.doc2bow(desc)
print(dictionary.doc2bow(content_data[0] [ :5]))
print(content_data[0] [ :5])

[(9, 1), (11, 1), (35, 1), (62, 1), (103, 1)]
['أكد', 'أمين', 'عام', 'وزاره', 'تربيه']


In [89]:
corpus = [dictionary.doc2bow(desc) for desc in content_data]

word_frequencies = [[(dictionary[id_], frequency) for id_, frequency in line] for line in corpus[0:3]]

print(word_frequencies)

[[('10', 1), ('15', 1), ('42', 1), ('600', 1), ('820', 1), ('أحتاج', 1), ('أحد', 1), ('أرض', 1), ('أشار', 1), ('أكد', 3), ('أمر', 1), ('أمين', 1), ('أن', 5), ('أنتظر', 1), ('أنتقل', 1), ('أنجز', 1), ('أو', 1), ('إجراء', 1), ('إداري', 1), ('إستراتيجي', 1), ('إعداد', 1), ('إلي', 6), ('ابن', 1), ('ارحاب', 1), ('استئجار', 1), ('اكتظاظ', 1), ('ال', 1), ('الذي', 3), ('القبيلات', 1), ('بناء', 7), ('بين', 1), ('تأكد', 1), ('تبرع', 2), ('تحديد', 2), ('تحويل', 1), ('تربيه', 5), ('تسلم', 1), ('تسليم', 1), ('تعلق', 1), ('تعليم', 1), ('تنسيق', 1), ('توزيع', 1), ('توسع', 1), ('جاء', 1), ('جديد', 1), ('جمعيه', 1), ('حاجه', 3), ('حسني', 1), ('حكومي', 2), ('خلال', 3), ('دراسه', 1), ('دكتور', 1), ('دوام', 1), ('زال', 1), ('زياده', 1), ('سبب', 1), ('سلامه', 1), ('سنه', 2), ('سواء', 1), ('شأن', 1), ('طالب', 2), ('طلب', 2), ('عام', 2), ('عدد', 1), ('علم', 1), ('علمي', 1), ('علي', 1), ('عمل', 1), ('فتره', 3), ('في', 5), ('فيما', 1), ('قبل', 1), ('قبيلات', 4), ('قد', 1), ('ل', 1), ('لكن', 1), ('ما', 1), ('ما

The above results shows vocabulary with their frequency.

### Build Tf-Idf and LSI Model

Tf-Idf means, Term frequency-Inverse Document Frequency. it is a commonly used NLP model that helps you determine the most important words in each document in the corpus. Once the Tf-Idf is build, pass it to LSI model and specify the num of features to build

In [97]:
%time SE_arabic_tfidf_model = models.TfidfModel(corpus, id2word=dictionary)
%time SE_arabic_lsi_model = models.LsiModel(SE_arabic_tfidf_model[corpus], id2word=dictionary, num_topics=300)

CPU times: total: 156 ms
Wall time: 147 ms
CPU times: total: 16.5 s
Wall time: 5.79 s


Serialize and Store the corpus locally for easy retrival whenver required.

In [98]:
%time corpora.MmCorpus.serialize('SE_arabic_tfidf_model_mm', SE_arabic_tfidf_model[corpus])
%time corpora.MmCorpus.serialize('SE_arabic_lsi_model_mm',SE_arabic_lsi_model[SE_arabic_tfidf_model[corpus]])

CPU times: total: 2.36 s
Wall time: 2.37 s
CPU times: total: 4.41 s
Wall time: 4.48 s


In [99]:
#Load the indexed corpus
SE_arabic_tfidf_corpus = corpora.MmCorpus('SE_arabic_tfidf_model_mm')
SE_arabic_lsi_corpus = corpora.MmCorpus('SE_arabic_lsi_model_mm')

print(SE_arabic_tfidf_corpus)
print(SE_arabic_lsi_corpus)


MmCorpus(5084 documents, 23922 features, 740285 non-zero entries)
MmCorpus(5084 documents, 300 features, 1499100 non-zero entries)


In [147]:
from gensim.similarities import MatrixSimilarity

%time SE_index = MatrixSimilarity(SE_arabic_lsi_corpus, num_features = SE_arabic_lsi_corpus.num_terms)

CPU times: total: 1.97 s
Wall time: 2 s


## Time for Semantic Search

Now comes the fun part. With the index of movies initialized and loaded, we can use it to find similar movies based


We will input a search query and model will return relevant movie titles with "Relevance %" which is the similarity score. The higher the similarity score, the more similar the query to the documetn at the given index

Below is the helper function to search the index, sort and return the results

In [103]:
# spacy_tokenizer

In [160]:
from operator import itemgetter

def search_similar_documents(search_term):
    print('Searching {}'.format(search_term))
    print()

    query_bow = dictionary.doc2bow(search_term.split())
    query_tfidf = SE_arabic_tfidf_model[query_bow]
    query_lsi = SE_arabic_lsi_model[query_tfidf]

    SE_index.num_best = 5

    SEs_list = SE_index[query_lsi]

    SEs_list.sort(key=itemgetter(1), reverse=True)
    SE_names = []

    for j, SE in enumerate(SEs_list):
        dict_entry = {
                'Relevance': round((SE[1] * 100),2),
                'Title': docs_df_cleaned['title'][SE[0]],
                'Content': docs_df_cleaned['content_clean'][SE[0]]
            }
        print("Title:  {}  (score: {})".format(dict_entry['Title'], dict_entry['Relevance']))
        print("(Content) {}".format(dict_entry['Content']))
        print()
        
        
        SE_names.append (
            dict_entry
        )
        if j == (SE_index.num_best-1):
            break
    
    return pd.DataFrame(SE_names, columns=['Relevance','Title','Content'])


In [170]:
# search for movie tiles that are related to below search parameters
search_similar_documents(clean_text2('ديوان ملكي'))

Searching ديوان ملك

Title:  حداد: عدم امتلاك ديوان المحاسبة الضابطة العدلية أضعف دوره  (score: 84.68)
(Content) حداد عدم امتلاك ديوان محاسبه ضابطه عدل ي أضعف دور قال رئيس ديوان محاسبه عاصم حداد أن عدم امتلاك ديوان محاسبه ضابطه عدل ي أضعف دور قدره علي حفاظ علي مال عام مشدد علي ضروره امتلاك أداه أو آلي إلزام مؤسسه الذي خضع رقابه أجاب ه علي استيضاح أضاف خلال لقاء في منتدي إعلام ي الذي نظم مركز حمايه حر صحف ي تعاون مع شركه زين مساء أمس أن غياب ضابطه عدل ي أفقد ديوان قوه لازم إجبار جهه الذي خضع ديوان محاسبه اتخاذ قرار تصويب ي لازم أو تحويل إلي هيئه نزاهه أو قضاء أشار إلي معاناه ديوان نقص كادر بشري سبب قرار حكومه إجراء إحاله علي تقاعد وقف تعيين نتيجه جائح كور لافت إلي أن ديوان عمل علي تعويض ذلك نقص من خلال تحول رقمي كلي أكد وجود عائق تحول دون عمل ديوان رغم تمتع ديوان أراد ه سياسي دعم ما خلل تطبيق أن كل خشي رقابه هي غير محبب قال أن بقاء أمر تعيين عزل رئيس ديوان محاسبه بتنسيب من رئيس وزير حلقه ضعف في عمل من ناحيه إداري رئيس ديوان محاسبه أرتبط رئيس وزير رقابي ؛ أرتبط ديوان المحاسبهبمجلس أمه لك

,Relevance,Title,Content
0,84.68,حداد: عدم امتلاك ديوان المحاسبة الضابطة العدلي...,حداد عدم امتلاك ديوان محاسبه ضابطه عدل ي أضعف ...
1,79.91,جائحة كورونا أوقفت ملء 120 شاغرا في ديوان المح...,جائح كور أوقف ملء 120 شاغر في ديوان محاسبه أعت...
2,76.82,حداد:لم يمنع ديوان المحاسبة من التدقيق على أعم...,حداد لم منع ديوان محاسبه من تدقيق علي عمل مجلس...
3,74.62,ديوان المحاسبة : مئات من القضايا التي حدثت بها...,ديوان محاسبه مئه من قضيه الذي حدث ب تجاوز إدار...
4,73.78,"""الخدمة المدنية"" يدعو مرشحين للامتحان (أسماء)",خدمه مدني دعا مرشح امتحان اسم دعا ديوان خدمه...


The model returns movie titles with "Relevance %". Definitely, the top list movies are related to crimes and drugs.